In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


In [2]:
# Charger le fichier CSV
file_path = 'OurData.csv'
df = pd.read_csv(file_path)
# Supprimer les colonnes 'Date' et 'Heure'
data = df.drop(["Date", "Heure"], axis=1)
# Convertir toutes les colonnes en float
data = data.astype(float)

FileNotFoundError: [Errno 2] No such file or directory: 'OurData.csv'

In [ ]:
data.isnull().sum()

RH         1081
AirTC      1081
G          1081
Rn_Avg     1081
Le_flux    1096
Ws         1097
Psat       1081
P_air      1081
VPD        1081
Rs            0
dtype: int64

In [ ]:
# Imputer les valeurs manquantes par la moyenne
imputer = SimpleImputer(strategy='mean')
# Remplacer les valeurs manquantes par la moyenne dans les colonnes spécifiées
colonnes_a_traiter = ['RH', 'AirTC', 'G', 'Rn_Avg', 'Le_flux', 'Ws', 'Psat', 'P_air', 'VPD', 'Rs']
data[colonnes_a_traiter] = data[colonnes_a_traiter].fillna(-9999999999999)

In [ ]:
data.isnull().sum()

RH         0
AirTC      0
G          0
Rn_Avg     0
Le_flux    0
Ws         0
Psat       0
P_air      0
VPD        0
Rs         0
dtype: int64

In [ ]:
# Séparer les données en entrées (features) et cible (target)
X = data.drop("Le_flux", axis=1)
y = data["Le_flux"]

In [ ]:
# Calculer la moyenne et l'écart-type des caractéristiques (features)
mean = X.mean(axis=0)
std = X.std(axis=0)

In [ ]:
# Normaliser les caractéristiques (features)
X -= mean
X /= std

In [ ]:
# Fractionner les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)
# Normaliser la cible (target)
mean_target = y_train.mean()
std_target = y_train.std()
y_train -= mean_target
y_train /= std_target
y_test -= mean_target
y_test /= std_target

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

# Create a KNN regressor
model = KNeighborsRegressor()

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:", best_params)

Mean Squared Error: 0.011187137863194375
Mean Absolute Error: 0.009686377906901472


In [ ]:
# Make predictions on the test set using the best model
predictions = best_model.predict(X_test)

# Print the predictions and corresponding actual values
for i in range(len(predictions)):
    print(f"Actual: {y_test.iloc[i]}, Predicted: {predictions[i]}")

Actual: 0.3466090101461702, Predicted: 0.3441045119614474
Actual: 0.34660901014616374, Predicted: 0.3545608889318365
Actual: -2.884744751488302, Predicted: -2.883096287776134
Actual: 0.34660901014616025, Predicted: 0.34230116215705686
Actual: 0.34660901014616374, Predicted: 0.332315200659933
Actual: 0.34660901014616374, Predicted: 0.3486252154300883
Actual: 0.34660901014616713, Predicted: 0.3506145438960614
Actual: 0.3466090101461712, Predicted: 0.3382398055240369
Actual: -2.884744751488302, Predicted: -2.885011141026801
Actual: 0.3466090101461655, Predicted: 0.3427422108760071
Actual: 0.34660901014616374, Predicted: 0.3547631498393307
Actual: 0.3466090101461645, Predicted: 0.34522299408165324
Actual: 0.3466090101461668, Predicted: 0.3420449660464978
Actual: 0.34660901014616374, Predicted: 0.3334635180282525
Actual: 0.34660901014616413, Predicted: 0.33095302591211917
Actual: 0.34660901014616374, Predicted: 0.34912318715381047
Actual: 0.34660901014616374, Predicted: 0.3599087879164005
A

In [ ]:
print(type(y_test))
print(type(predictions))


<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [ ]:
import pickle

# Load the saved model
